In [ ]:
!pip install gradio

In [6]:
import gradio as gr
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

# Create and train a model with sample data
def create_and_train_model():
    # Create realistic sample data based on typical customer churn patterns
    np.random.seed(42)
    n_samples = 2000

    # Generate realistic features
    data = {
        'gender': np.random.randint(0, 2, n_samples),
        'SeniorCitizen': np.random.randint(0, 2, n_samples),
        'Partner': np.random.randint(0, 2, n_samples),
        'Dependents': np.random.randint(0, 2, n_samples),
        'PhoneService': np.random.randint(0, 2, n_samples),
        'MultipleLines': np.random.randint(0, 2, n_samples),
        'InternetService': np.random.randint(0, 3, n_samples),
        'OnlineSecurity': np.random.randint(0, 3, n_samples),
        'OnlineBackup': np.random.randint(0, 3, n_samples),
        'DeviceProtection': np.random.randint(0, 3, n_samples),
        'TechSupport': np.random.randint(0, 3, n_samples),
        'StreamingTV': np.random.randint(0, 2, n_samples),
        'StreamingMovies': np.random.randint(0, 2, n_samples),
        'Contract': np.random.randint(0, 3, n_samples),
        'PaperlessBilling': np.random.randint(0, 2, n_samples),
        'PaymentMethod': np.random.randint(0, 4, n_samples),
        'MonthlyCharges': np.random.uniform(20, 120, n_samples),
        'TotalCharges': np.random.uniform(100, 5000, n_samples),
        'tenure_group': np.random.randint(0, 6, n_samples)
    }

    # Create target variable with realistic churn patterns
    # Higher churn probability for: high monthly charges, month-to-month contracts, fiber optic users
    X = pd.DataFrame(data)
    y = np.zeros(n_samples)

    # Business rules for churn
    churn_prob = (
        (X['MonthlyCharges'] > 70) * 0.3 +
        (X['Contract'] == 0) * 0.4 +  # Month-to-month
        (X['InternetService'] == 1) * 0.2 +  # Fiber optic
        (X['PaymentMethod'] == 0) * 0.1  # Electronic check
    )

    y = (np.random.random(n_samples) < churn_prob).astype(int)

    # Train the model
    model = GradientBoostingClassifier(n_estimators=100, random_state=42, max_depth=5)
    model.fit(X, y)

    # Calculate accuracy
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    print(f"Model trained with accuracy: {accuracy:.2f}")

    return model

# Create and save the model
model = create_and_train_model()

# Save the model for future use
with open('churn_model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Model created and saved successfully!")

# Function to preprocess input data
def preprocess_input(data):
    # Convert input data to DataFrame
    df = pd.DataFrame(data, index=[0])

    # Convert categorical variables to numeric (matching our training)
    internet_mapping = {'DSL': 0, 'Fiber optic': 1, 'No': 2}
    contract_mapping = {'Month-to-month': 0, 'One year': 1, 'Two year': 2}
    payment_mapping = {
        'Electronic check': 0,
        'Mailed check': 1,
        'Bank transfer (automatic)': 2,
        'Credit card (automatic)': 3
    }

    df['InternetService'] = df['InternetService'].map(internet_mapping)
    df['Contract'] = df['Contract'].map(contract_mapping)
    df['PaymentMethod'] = df['PaymentMethod'].map(payment_mapping)

    return df

# Prediction function with clear user feedback
def predict_churn(gender, senior_citizen, partner, dependents, phone_service,
                 multiple_lines, internet_service, online_security, online_backup,
                 device_protection, tech_support, streaming_tv, streaming_movies,
                 contract, paperless_billing, payment_method, monthly_charges,
                 total_charges, tenure_group):

    # Create dictionary from user inputs
    user_data = {
        'gender': gender,
        'SeniorCitizen': senior_citizen,
        'Partner': partner,
        'Dependents': dependents,
        'PhoneService': phone_service,
        'MultipleLines': multiple_lines,
        'InternetService': internet_service,
        'OnlineSecurity': online_security,
        'OnlineBackup': online_backup,
        'DeviceProtection': device_protection,
        'TechSupport': tech_support,
        'StreamingTV': streaming_tv,
        'StreamingMovies': streaming_movies,
        'Contract': contract,
        'PaperlessBilling': paperless_billing,
        'PaymentMethod': payment_method,
        'MonthlyCharges': monthly_charges,
        'TotalCharges': total_charges,
        'tenure_group': tenure_group
    }

    try:
        # Preprocess input data
        processed_data = preprocess_input(user_data)

        # Make prediction
        prediction = model.predict(processed_data)
        prediction_proba = model.predict_proba(processed_data)

        churn_probability = prediction_proba[0][1] * 100  # Probability of churn

        # Detailed result message
        if prediction[0] == 1:
            result = f"🚨 HIGH RISK OF CHURN ({churn_probability:.1f}% probability)\n\n"
            result += "• Customer is likely to leave\n"
            result += "• Consider retention strategies\n"
            result += "• Review service quality and pricing"
        else:
            result = f"✅ LOW RISK OF CHURN ({churn_probability:.1f}% probability)\n\n"
            result += "• Customer is likely to stay\n"
            result += "• Good customer relationship\n"
            result += "• Continue current service quality"

        return result

    except Exception as e:
        return f"❌ Error in prediction: {str(e)}"

# Create user-friendly Gradio interface
with gr.Blocks(title="Customer Churn Predictor", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🎯 Customer Churn Prediction Dashboard")
    gr.Markdown("Predict whether a customer is likely to leave your service")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 👤 Personal Information")
            gender = gr.Radio(label="Gender", choices=[0, 1], value=0,
                            info="0 = Female, 1 = Male")
            senior_citizen = gr.Radio(label="Senior Citizen", choices=[0, 1], value=0)
            partner = gr.Radio(label="Has Partner", choices=[0, 1], value=0)
            dependents = gr.Radio(label="Has Dependents", choices=[0, 1], value=0)

        with gr.Column(scale=1):
            gr.Markdown("### 📞 Service Details")
            phone_service = gr.Radio(label="Phone Service", choices=[0, 1], value=1)
            multiple_lines = gr.Radio(label="Multiple Lines", choices=[0, 1], value=0)
            internet_service = gr.Dropdown(label="Internet Service",
                                         choices=['DSL', 'Fiber optic', 'No'],
                                         value='DSL')

        with gr.Column(scale=1):
            gr.Markdown("### 🔒 Service Features")
            online_security = gr.Slider(label="Online Security", minimum=0, maximum=2, step=1, value=1,
                                      info="0 = No, 1 = Yes, 2 = No internet service")
            online_backup = gr.Slider(label="Online Backup", minimum=0, maximum=2, step=1, value=1,
                                    info="0 = No, 1 = Yes, 2 = No internet service")
            device_protection = gr.Slider(label="Device Protection", minimum=0, maximum=2, step=1, value=1,
                                        info="0 = No, 1 = Yes, 2 = No internet service")
            tech_support = gr.Slider(label="Tech Support", minimum=0, maximum=2, step=1, value=1,
                                   info="0 = No, 1 = Yes, 2 = No internet service")

        with gr.Column(scale=1):
            gr.Markdown("### 📺 Streaming Services")
            streaming_tv = gr.Radio(label="Streaming TV", choices=[0, 1], value=0)
            streaming_movies = gr.Radio(label="Streaming Movies", choices=[0, 1], value=0)

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📝 Contract & Billing")
            contract = gr.Dropdown(label="Contract Type",
                                 choices=['Month-to-month', 'One year', 'Two year'],
                                 value='Month-to-month')
            paperless_billing = gr.Radio(label="Paperless Billing", choices=[0, 1], value=1)
            payment_method = gr.Dropdown(label="Payment Method",
                                       choices=['Electronic check', 'Mailed check',
                                               'Bank transfer (automatic)', 'Credit card (automatic)'],
                                       value='Electronic check')

        with gr.Column(scale=1):
            gr.Markdown("### 💰 Financial Information")
            monthly_charges = gr.Slider(label="Monthly Charges ($)", minimum=20, maximum=120, value=65.0, step=0.5)
            total_charges = gr.Slider(label="Total Charges ($)", minimum=100, maximum=5000, value=2000.0, step=10.0)
            tenure_group = gr.Slider(label="Tenure Group (months)", minimum=0, maximum=60, value=24, step=1,
                                   info="Customer loyalty duration")

    with gr.Row():
        predict_btn = gr.Button("🔮 Predict Churn Risk", variant="primary", size="lg")
        clear_btn = gr.Button("🔄 Clear Inputs", variant="secondary")

    with gr.Row():
        output = gr.Textbox(label="Prediction Result", lines=6, max_lines=10)

    # Set up button actions
    predict_btn.click(
        fn=predict_churn,
        inputs=[gender, senior_citizen, partner, dependents, phone_service,
               multiple_lines, internet_service, online_security, online_backup,
               device_protection, tech_support, streaming_tv, streaming_movies,
               contract, paperless_billing, payment_method, monthly_charges,
               total_charges, tenure_group],
        outputs=output
    )

    def clear_inputs():
        return [0, 0, 0, 0, 1, 0, 'DSL', 1, 1, 1, 1, 0, 0, 'Month-to-month', 1,
                'Electronic check', 65.0, 2000.0, 24]

    clear_btn.click(
        fn=clear_inputs,
        outputs=[gender, senior_citizen, partner, dependents, phone_service,
                multiple_lines, internet_service, online_security, online_backup,
                device_protection, tech_support, streaming_tv, streaming_movies,
                contract, paperless_billing, payment_method, monthly_charges,
                total_charges, tenure_group]
    )

# Launch the app
print("🚀 Starting Customer Churn Prediction App...")
demo.launch(share=True, debug=True)

Model trained with accuracy: 0.94
Model created and saved successfully!
🚀 Starting Customer Churn Prediction App...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e2e18b888f83e35bf7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://507256cc363866d33d.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://e2e18b888f83e35bf7.gradio.live


In [8]:
import gradio as gr
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# --------------------------
# 1. Train a Simple Model
# --------------------------
def train_churn_model():
    np.random.seed(42)
    n_samples = 2000

    # Minimal feature set for simplicity
    data = {
        "MonthlyCharges": np.random.uniform(20, 120, n_samples),
        "Contract": np.random.randint(0, 3, n_samples),  # 0=Month-to-month,1=1yr,2=2yr
        "InternetService": np.random.randint(0, 3, n_samples), # 0=DSL,1=Fiber,2=None
        "Tenure": np.random.randint(0, 60, n_samples),  # months
    }

    X = pd.DataFrame(data)

    # Target variable: churn probability
    churn_prob = (
        (X["MonthlyCharges"] > 70) * 0.3 +
        (X["Contract"] == 0) * 0.4 +  # month-to-month
        (X["InternetService"] == 1) * 0.2  # fiber
    )
    y = (np.random.rand(n_samples) < churn_prob).astype(int)

    model = GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=42)
    model.fit(X, y)

    acc = accuracy_score(y, model.predict(X))
    print(f"Model trained. Accuracy (on training data): {acc:.2f}")

    return model

model = train_churn_model()

# --------------------------
# 2. Prediction Function
# --------------------------
def predict_churn(monthly_charges, contract, internet_service, tenure):
    input_df = pd.DataFrame([{
        "MonthlyCharges": monthly_charges,
        "Contract": {"Month-to-month": 0, "One year": 1, "Two year": 2}[contract],
        "InternetService": {"DSL": 0, "Fiber optic": 1, "No": 2}[internet_service],
        "Tenure": tenure
    }])

    proba = model.predict_proba(input_df)[0][1] * 100
    prediction = model.predict(input_df)[0]

    if prediction == 1:
        return f"🚨 High Churn Risk ({proba:.1f}%). Suggest retention strategies!"
    else:
        return f"✅ Low Churn Risk ({proba:.1f}%). Customer likely to stay."

# --------------------------
# 3. Gradio UI
# --------------------------
with gr.Blocks(title="Customer Churn Predictor") as demo:
    gr.Markdown("## 🎯 Customer Churn Prediction (Simple & Clean)")
    gr.Markdown("Enter customer details to predict churn risk.")

    with gr.Row():
        monthly_charges = gr.Slider(20, 120, value=60, step=1, label="💰 Monthly Charges ($)")
        contract = gr.Dropdown(["Month-to-month", "One year", "Two year"], value="Month-to-month", label="📜 Contract")
        internet_service = gr.Dropdown(["DSL", "Fiber optic", "No"], value="DSL", label="🌐 Internet Service")
        tenure = gr.Slider(0, 60, value=24, step=1, label="📆 Tenure (months)")

    predict_btn = gr.Button("🔮 Predict")
    output = gr.Textbox(label="Prediction", lines=2)

    predict_btn.click(
        fn=predict_churn,
        inputs=[monthly_charges, contract, internet_service, tenure],
        outputs=output
    )

print("🚀 App starting...")
demo.launch(share=True)


Model trained. Accuracy (on training data): 0.89
🚀 App starting...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://94e8fc003639de85a3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [9]:
import gradio as gr
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# --------------------------
# 1. Train Model
# --------------------------
def train_churn_model():
    np.random.seed(42)
    n_samples = 2000

    data = {
        "MonthlyCharges": np.random.uniform(20, 120, n_samples),
        "Contract": np.random.randint(0, 3, n_samples),
        "InternetService": np.random.randint(0, 3, n_samples),
        "Tenure": np.random.randint(0, 60, n_samples),
    }

    X = pd.DataFrame(data)

    churn_prob = (
        (X["MonthlyCharges"] > 70) * 0.3 +
        (X["Contract"] == 0) * 0.4 +
        (X["InternetService"] == 1) * 0.2
    )
    y = (np.random.rand(n_samples) < churn_prob).astype(int)

    model = GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=42)
    model.fit(X, y)

    acc = accuracy_score(y, model.predict(X))
    print(f"Model trained. Accuracy (on training data): {acc:.2f}")

    return model

model = train_churn_model()

# --------------------------
# 2. Prediction Function
# --------------------------
def predict_churn(monthly_charges, contract, internet_service, tenure):
    input_df = pd.DataFrame([{
        "MonthlyCharges": monthly_charges,
        "Contract": {"Month-to-month": 0, "One year": 1, "Two year": 2}[contract],
        "InternetService": {"DSL": 0, "Fiber optic": 1, "No": 2}[internet_service],
        "Tenure": tenure
    }])

    proba = model.predict_proba(input_df)[0][1] * 100
    prediction = model.predict(input_df)[0]

    if prediction == 1:
        msg = f"🚨 High Churn Risk ({proba:.1f}%). Suggest retention strategies!"
    else:
        msg = f"✅ Low Churn Risk ({proba:.1f}%). Customer likely to stay."

    # ---- Generate probability trend chart ----
    charges_range = np.linspace(20, 120, 100)
    probs = []
    for c in charges_range:
        df = input_df.copy()
        df["MonthlyCharges"] = c
        probs.append(model.predict_proba(df)[0][1] * 100)

    fig, ax = plt.subplots()
    ax.plot(charges_range, probs, label="Churn Probability")
    ax.axvline(monthly_charges, color="red", linestyle="--", label="Your Input")
    ax.set_xlabel("Monthly Charges ($)")
    ax.set_ylabel("Churn Probability (%)")
    ax.set_title("📊 Churn Probability vs Monthly Charges")
    ax.legend()
    plt.tight_layout()

    return msg, fig

# --------------------------
# 3. Gradio UI
# --------------------------
with gr.Blocks(title="Customer Churn Predictor") as demo:
    gr.Markdown("## 🎯 Customer Churn Prediction (Interactive)")
    gr.Markdown("Enter details below to predict churn risk. Notes included for guidance 👇")

    with gr.Row():
        monthly_charges = gr.Slider(
            20, 120, value=60, step=1, label="💰 Monthly Charges ($)",
            info="Higher charges often → more likely to churn"
        )
        contract = gr.Dropdown(
            ["Month-to-month", "One year", "Two year"], value="Month-to-month", label="📜 Contract",
            info="Month-to-month = higher churn risk"
        )
        internet_service = gr.Dropdown(
            ["DSL", "Fiber optic", "No"], value="DSL", label="🌐 Internet Service",
            info="Fiber optic users churn more often"
        )
        tenure = gr.Slider(
            0, 60, value=24, step=1, label="📆 Tenure (months)",
            info="Longer tenure → more loyal customer"
        )

    predict_btn = gr.Button("🔮 Predict")
    output_text = gr.Textbox(label="Prediction", lines=2)
    output_chart = gr.Plot(label="Churn Probability Chart")

    predict_btn.click(
        fn=predict_churn,
        inputs=[monthly_charges, contract, internet_service, tenure],
        outputs=[output_text, output_chart]
    )

print("🚀 App starting...")
demo.launch(share=True)


Model trained. Accuracy (on training data): 0.89
🚀 App starting...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://73c7db7e3ba161d9f5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [11]:
import gradio as gr
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# --------------------------
# 1. Train Model + Create Sample Data
# --------------------------
def create_data_and_train():
    np.random.seed(42)
    n_samples = 2000

    data = {
        "MonthlyCharges": np.random.uniform(20, 120, n_samples),
        "Contract": np.random.randint(0, 3, n_samples),
        "InternetService": np.random.randint(0, 3, n_samples),
        "Tenure": np.random.randint(0, 60, n_samples),
    }
    df = pd.DataFrame(data)

    churn_prob = (
        (df["MonthlyCharges"] > 70) * 0.3 +
        (df["Contract"] == 0) * 0.4 +
        (df["InternetService"] == 1) * 0.2
    )
    y = (np.random.rand(n_samples) < churn_prob).astype(int)
    df["Churn"] = y

    model = GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=42)
    model.fit(df.drop(columns="Churn"), y)

    acc = accuracy_score(y, model.predict(df.drop(columns="Churn")))
    print(f"Model trained. Accuracy (on training data): {acc:.2f}")

    return model, df

model, sample_data = create_data_and_train()
sample_csv_path = "sample_churn_data.csv"
sample_data.to_csv(sample_csv_path, index=False)

# --------------------------
# 2. Prediction Function
# --------------------------
def predict_churn(monthly_charges, contract, internet_service, tenure):
    input_df = pd.DataFrame([{
        "MonthlyCharges": monthly_charges,
        "Contract": {"Month-to-month": 0, "One year": 1, "Two year": 2}[contract],
        "InternetService": {"DSL": 0, "Fiber optic": 1, "No": 2}[internet_service],
        "Tenure": tenure
    }])

    proba = model.predict_proba(input_df)[0][1] * 100
    prediction = model.predict(input_df)[0]

    msg = (
        f"🚨 High Churn Risk ({proba:.1f}%). Suggest retention strategies!"
        if prediction == 1 else
        f"✅ Low Churn Risk ({proba:.1f}%). Customer likely to stay."
    )

    return msg, proba

# --------------------------
# 3. Random Example Function
# --------------------------
def random_example():
    row = sample_data.sample(1).iloc[0]
    contract_map = {0: "Month-to-month", 1: "One year", 2: "Two year"}
    internet_map = {0: "DSL", 1: "Fiber optic", 2: "No"}
    return (
        float(row["MonthlyCharges"]),
        contract_map[row["Contract"]],
        internet_map[row["InternetService"]],
        int(row["Tenure"])
    )

# --------------------------
# 4. Compare Function
# --------------------------
def compare_customers(mc1, c1, net1, t1, mc2, c2, net2, t2):
    msg1, p1 = predict_churn(mc1, c1, net1, t1)
    msg2, p2 = predict_churn(mc2, c2, net2, t2)

    comparison = f"""
    👤 **Customer A:** {msg1}
    👤 **Customer B:** {msg2}

    📊 Difference in Churn Risk: {abs(p1 - p2):.1f}%
    """

    return comparison

# --------------------------
# 5. Gradio UI
# --------------------------
with gr.Blocks(title="Customer Churn Predictor") as demo:
    gr.Markdown("## 🎯 Customer Churn Prediction Dashboard")
    gr.Markdown("Predict churn risk, view charts, test random customers, and compare two profiles side-by-side.")

    # ----------------- Single Prediction -----------------
    gr.Markdown("### 🔍 Single Customer Prediction")
    with gr.Row():
        monthly_charges = gr.Slider(20, 120, value=60, step=1, label="💰 Monthly Charges ($)",
                                    info="Higher charges → more churn risk")
        contract = gr.Dropdown(["Month-to-month", "One year", "Two year"], value="Month-to-month",
                               label="📜 Contract", info="Month-to-month = higher churn")
        internet_service = gr.Dropdown(["DSL", "Fiber optic", "No"], value="DSL",
                                       label="🌐 Internet Service", info="Fiber optic users churn more")
        tenure = gr.Slider(0, 60, value=24, step=1, label="📆 Tenure (months)",
                           info="Longer tenure → more loyalty")

    with gr.Row():
        predict_btn = gr.Button("🔮 Predict")
        random_btn = gr.Button("🎲 Random Example Customer")

    output_text = gr.Textbox(label="Prediction", lines=2)

    predict_btn.click(
        fn=lambda mc, c, net, t: predict_churn(mc, c, net, t)[0],
        inputs=[monthly_charges, contract, internet_service, tenure],
        outputs=output_text
    )

    random_btn.click(
        fn=random_example,
        inputs=[],
        outputs=[monthly_charges, contract, internet_service, tenure]
    )

    # ----------------- Compare Mode -----------------
    gr.Markdown("---")
    gr.Markdown("### ⚖️ Compare Two Customers")

    with gr.Row():
        with gr.Column():
            gr.Markdown("**Customer A**")
            mc1 = gr.Slider(20, 120, value=55, step=1, label="Monthly Charges A")
            c1 = gr.Dropdown(["Month-to-month", "One year", "Two year"], value="One year", label="Contract A")
            net1 = gr.Dropdown(["DSL", "Fiber optic", "No"], value="DSL", label="Internet A")
            t1 = gr.Slider(0, 60, value=12, step=1, label="Tenure A (months)")
        with gr.Column():
            gr.Markdown("**Customer B**")
            mc2 = gr.Slider(20, 120, value=85, step=1, label="Monthly Charges B")
            c2 = gr.Dropdown(["Month-to-month", "One year", "Two year"], value="Month-to-month", label="Contract B")
            net2 = gr.Dropdown(["DSL", "Fiber optic", "No"], value="Fiber optic", label="Internet B")
            t2 = gr.Slider(0, 60, value=6, step=1, label="Tenure B (months)")

    compare_btn = gr.Button("⚖️ Compare Customers")
    compare_output = gr.Textbox(label="Comparison Result", lines=4)

    compare_btn.click(
        fn=compare_customers,
        inputs=[mc1, c1, net1, t1, mc2, c2, net2, t2],
        outputs=compare_output
    )

    # ----------------- Download -----------------
    gr.Markdown("---")
    gr.Markdown("### 📂 Download Sample Data")
    gr.File(value=sample_csv_path, label="Download Synthetic Churn Dataset")

print("🚀 App starting...")
demo.launch(share=True)


Model trained. Accuracy (on training data): 0.89
🚀 App starting...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://17bfc8349e5e603f1d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/IPython/core/events.py:89: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  func(*args, **kwargs)


In [ ]:
pip install gradio scikit-learn matplotlib pandas numpy


In [15]:
import gradio as gr
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import json
import time

# --------------------------
# 1) DATA + MODEL
# --------------------------

def create_data_and_train(n_samples: int = 2000, seed: int = 42):
    """Create a small synthetic churn dataset and train a simple model.
    Features kept intentionally minimal for clarity + maintenance.
    """
    rng = np.random.default_rng(seed)

    data = {
        "MonthlyCharges": rng.uniform(20, 120, n_samples),          # $ per month
        "Contract": rng.integers(0, 3, n_samples),                  # 0=Month-to-month, 1=One year, 2=Two year
        "InternetService": rng.integers(0, 3, n_samples),           # 0=DSL, 1=Fiber optic, 2=No
        "Tenure": rng.integers(0, 60, n_samples),                   # months
    }
    df = pd.DataFrame(data)

    # Churn probability business rules (intentionally simple and readable)
    churn_prob = (
        (df["MonthlyCharges"] > 70) * 0.30 +
        (df["Contract"] == 0) * 0.40 +      # Month-to-month churns more
        (df["InternetService"] == 1) * 0.20  # Fiber optic churns more in this toy setup
    )
    y = (np.random.default_rng(seed + 7).random(n_samples) < churn_prob).astype(int)
    df["Churn"] = y

    model = GradientBoostingClassifier(n_estimators=120, max_depth=5, random_state=seed)
    model.fit(df[["MonthlyCharges", "Contract", "InternetService", "Tenure"]], y)

    acc = accuracy_score(y, model.predict(df[["MonthlyCharges", "Contract", "InternetService", "Tenure"]]))
    return model, df, acc

model, sample_data, train_acc = create_data_and_train()
SAMPLE_CSV_PATH = "sample_churn_data.csv"
sample_data.to_csv(SAMPLE_CSV_PATH, index=False)

# Helpful mappings (kept in one place)
CONTRACT_TO_INT = {"Month-to-month": 0, "One year": 1, "Two year": 2}
INT_TO_CONTRACT = {v: k for k, v in CONTRACT_TO_INT.items()}
INTERNET_TO_INT = {"DSL": 0, "Fiber optic": 1, "No": 2}
INT_TO_INTERNET = {v: k for k, v in INTERNET_TO_INT.items()}

# --------------------------
# 2) CORE HELPERS
# --------------------------

def _predict_proba(monthly_charges: float, contract: str, internet_service: str, tenure: int) -> float:
    X = pd.DataFrame([{
        "MonthlyCharges": monthly_charges,
        "Contract": CONTRACT_TO_INT[contract],
        "InternetService": INTERNET_TO_INT[internet_service],
        "Tenure": tenure,
    }])
    return float(model.predict_proba(X)[0, 1]) * 100.0


def _trend_figure(monthly_charges: float, contract: str, internet_service: str, tenure: int, label: str = "Your Input"):
    charges_range = np.linspace(20, 120, 120)
    probs = []
    for c in charges_range:
        probs.append(_predict_proba(c, contract, internet_service, tenure))

    fig, ax = plt.subplots()
    ax.plot(charges_range, probs, label="Churn Probability vs Monthly Charges")
    ax.axvline(monthly_charges, linestyle="--", label=label)
    ax.set_xlabel("Monthly Charges ($)")
    ax.set_ylabel("Churn Probability (%)")
    ax.set_title("Churn Probability Trend")
    ax.legend()
    fig.tight_layout()
    return fig


def _dual_trend_figure(a, b):
    # a/b are dicts with keys: monthly_charges, contract, internet_service, tenure, label
    charges_range = np.linspace(20, 120, 120)
    probs_a, probs_b = [], []
    for c in charges_range:
        probs_a.append(_predict_proba(c, a["contract"], a["internet_service"], a["tenure"]))
        probs_b.append(_predict_proba(c, b["contract"], b["internet_service"], b["tenure"]))

    fig, ax = plt.subplots()
    ax.plot(charges_range, probs_a, label=f"A Trend ({a['label']})")
    ax.plot(charges_range, probs_b, label=f"B Trend ({b['label']})")
    ax.axvline(a["monthly_charges"], linestyle="--", label="A charges")
    ax.axvline(b["monthly_charges"], linestyle="--", label="B charges")
    ax.set_xlabel("Monthly Charges ($)")
    ax.set_ylabel("Churn Probability (%)")
    ax.set_title("Compare: Churn Probability vs Monthly Charges")
    ax.legend()
    fig.tight_layout()
    return fig

# --------------------------
# 3) GRADIO CALLBACKS
# --------------------------

def predict_single(monthly_charges, contract, internet_service, tenure):
    proba = _predict_proba(monthly_charges, contract, internet_service, tenure)
    label = "🚨 High Churn Risk" if proba >= 50 else "✅ Low Churn Risk"
    msg = f"{label} ({proba:.1f}%)."
    fig = _trend_figure(monthly_charges, contract, internet_service, tenure, label="Selected Charges")

    # Prepare a JSON export (as string). In UI we allow user to download.
    payload = {
        "timestamp": int(time.time()),
        "inputs": {
            "MonthlyCharges": monthly_charges,
            "Contract": contract,
            "InternetService": internet_service,
            "Tenure": tenure,
        },
        "prediction": {
            "churn_probability_percent": round(proba, 2),
            "label": "High" if proba >= 50 else "Low",
        },
        "model": {
            "estimator": "GradientBoostingClassifier",
            "train_accuracy": round(float(train_acc), 3),
            "note": "Toy synthetic model for demo; not for production."
        },
    }
    json_path = "single_prediction.json"
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, indent=2)

    return msg, fig, json_path


def random_example():
    row = sample_data.sample(1).iloc[0]
    return (
        float(row["MonthlyCharges"]),
        INT_TO_CONTRACT[int(row["Contract"])],
        INT_TO_INTERNET[int(row["InternetService"])],
        int(row["Tenure"]),
    )


def reset_defaults():
    return (60.0, "Month-to-month", "DSL", 24)


def compare_customers(mc1, c1, net1, t1, mc2, c2, net2, t2):
    p1 = _predict_proba(mc1, c1, net1, t1)
    p2 = _predict_proba(mc2, c2, net2, t2)

    msg1 = ("🚨 High" if p1 >= 50 else "✅ Low") + f" ({p1:.1f}%)"
    msg2 = ("🚨 High" if p2 >= 50 else "✅ Low") + f" ({p2:.1f}%)"
    diff = abs(p1 - p2)

    comparison_text = (
        f"👤 Customer A: {msg1}\n"
        f"👤 Customer B: {msg2}\n\n"
        f"📊 Difference in Churn Risk: {diff:.1f}%"
    )

    a = {"monthly_charges": mc1, "contract": c1, "internet_service": net1, "tenure": t1, "label": "A"}
    b = {"monthly_charges": mc2, "contract": c2, "internet_service": net2, "tenure": t2, "label": "B"}
    fig = _dual_trend_figure(a, b)

    # Export as JSON
    payload = {
        "timestamp": int(time.time()),
        "customer_A": {"MonthlyCharges": mc1, "Contract": c1, "InternetService": net1, "Tenure": t1, "probability_percent": round(p1, 2)},
        "customer_B": {"MonthlyCharges": mc2, "Contract": c2, "InternetService": net2, "Tenure": t2, "probability_percent": round(p2, 2)},
        "difference_percent": round(diff, 2),
    }
    json_path = "comparison_result.json"
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, indent=2)

    return comparison_text, fig, json_path

# --------------------------
# 4) UI
# --------------------------
with gr.Blocks(title="Customer Churn Predictor — BA Dashboard", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🎯 Customer Churn Prediction
    **What to enter:**
    - **💰 Monthly Charges ($):** 20 to 120. *Higher charges → usually higher churn risk.*
    - **📜 Contract:** Month-to-month / One year / Two year. *Month-to-month churns more.*
    - **🌐 Internet Service:** DSL / Fiber optic / No. *Fiber shows higher churn in this toy data.*
    - **📆 Tenure (months):** 0 to 60. *Longer tenure → more loyal.*
    """)

    with gr.Accordion("ℹ️ Model Card / Notes", open=False):
        gr.Markdown(
            f"""
            **Model:** GradientBoostingClassifier
            **Training accuracy (on synthetic data):** {train_acc:.2f}
            **Disclaimer:** This is a synthetic demo model. Do **not** use for real decisions.
            **Encodings:** Contract → 0 M2M, 1 One-year, 2 Two-year. Internet → 0 DSL, 1 Fiber, 2 No.
            """
        )

    with gr.Tabs():
        with gr.Tab("🔍 Single Prediction"):
            with gr.Row():
                monthly_charges = gr.Slider(20, 120, value=60, step=1, label="💰 Monthly Charges ($)", info="Higher charges → more likely to churn")
                contract = gr.Dropdown(["Month-to-month", "One year", "Two year"], value="Month-to-month", label="📜 Contract", info="Month-to-month = higher churn risk")
                internet_service = gr.Dropdown(["DSL", "Fiber optic", "No"], value="DSL", label="🌐 Internet Service", info="Fiber optic users churn more often in this toy dataset")
                tenure = gr.Slider(0, 60, value=24, step=1, label="📆 Tenure (months)", info="Longer tenure → more loyalty")

            with gr.Row():
                predict_btn = gr.Button("🔮 Predict", variant="primary")
                random_btn = gr.Button("🎲 Random Example")
                reset_btn = gr.Button("♻️ Reset")

            output_text = gr.Textbox(label="Prediction", lines=2)
            output_plot = gr.Plot(label="Churn Probability Chart")

            with gr.Row():
                gr.Markdown("**Export last prediction as JSON:**")
                single_json = gr.File(label="single_prediction.json")

            predict_btn.click(
                fn=predict_single,
                inputs=[monthly_charges, contract, internet_service, tenure],
                outputs=[output_text, output_plot, single_json]
            )
            random_btn.click(fn=random_example, inputs=[], outputs=[monthly_charges, contract, internet_service, tenure])
            reset_btn.click(fn=reset_defaults, inputs=[], outputs=[monthly_charges, contract, internet_service, tenure])

        with gr.Tab("⚖️ Compare Two Customers"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("**Customer A**")
                    mc1 = gr.Slider(20, 120, value=55, step=1, label="Monthly Charges A")
                    c1 = gr.Dropdown(["Month-to-month", "One year", "Two year"], value="One year", label="Contract A")
                    net1 = gr.Dropdown(["DSL", "Fiber optic", "No"], value="DSL", label="Internet A")
                    t1 = gr.Slider(0, 60, value=12, step=1, label="Tenure A (months)")
                with gr.Column():
                    gr.Markdown("**Customer B**")
                    mc2 = gr.Slider(20, 120, value=85, step=1, label="Monthly Charges B")
                    c2 = gr.Dropdown(["Month-to-month", "One year", "Two year"], value="Month-to-month", label="Contract B")
                    net2 = gr.Dropdown(["DSL", "Fiber optic", "No"], value="Fiber optic", label="Internet B")
                    t2 = gr.Slider(0, 60, value=6, step=1, label="Tenure B (months)")

            compare_btn = gr.Button("⚖️ Compare", variant="primary")
            compare_text = gr.Textbox(label="Comparison Result", lines=4)
            compare_plot = gr.Plot(label="Comparison Chart")
            with gr.Row():
                gr.Markdown("**Export comparison as JSON:**")
                compare_json = gr.File(label="comparison_result.json")

            compare_btn.click(
                fn=compare_customers,
                inputs=[mc1, c1, net1, t1, mc2, c2, net2, t2],
                outputs=[compare_text, compare_plot, compare_json]
            )

    gr.Markdown("---")
    gr.Markdown("### 📂 Download the Synthetic Training Data")
    gr.File(value=SAMPLE_CSV_PATH, label="sample_churn_data.csv")

if __name__ == "__main__":
    print("🚀 App starting...")
    demo.launch(share=True, debug=True)


🚀 App starting...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8007f6ffc91f08facf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7866 <> https://8007f6ffc91f08facf.gradio.live
